This codebook trains and fine-tunes a BERT model to predict moral sentiment  
This can be very slow depending on hardware.  
We used a v100, 32GB of RAM, 8 CPUS  
However, this code should be able to run on a system with 16GB of RAM, a dedicated GPU (we tested it on a RTX 2070s), and a 6-core CPU (e.g., Ryzen 5 3600)

## Load Packages

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model 
from keras.metrics import Precision, Recall

import pandas as pd
import numpy as np
import pickle as pkl
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from nltk.corpus import stopwords
import tokenization

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import nltk
nltk.download('stopwords')

foundations = {"mfrc":  {
                    "complete": ["care", "harm", "equality", "proportionality", "loyalty", "betrayal", "authority", "subversion", "purity", "degradation", "thin morality", "non-moral"],
                    "binding": ["individual", "binding", "proportionality", "thin morality", "non-moral"], 
                    "moral": ["moral", "thin morality", "non-moral"],
                    "full": ["care", "proportionality", "loyalty", "authority", "purity", "equality", "thin morality", "non-moral"]
               }
              }
classes = {"mfrc": {"full": 8, "moral": 3, "binding": 5, "complete": 12}}
activation = {"full": "sigmoid", "moral": "sigmoid", "binding": "sigmoid"}

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabdurah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Functions for training

In [8]:
def build_model(bert_layer, max_len=512, classes = 5, activation = "sigmoid"):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    outputs= bert_layer(dict(input_word_ids=input_word_ids,
    input_mask=input_mask,
    input_type_ids=segment_ids))

    sequence_output=outputs["sequence_output"]

    clf_output = sequence_output[:, 0, :]
    out = tf.keras.layers.Dense(classes, activation=activation)(clf_output)

    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='binary_crossentropy', metrics=[Precision(), Recall()])
    model.summary()
    return model

def get_binary(_y, threshold):
    y = _y.copy()
    y[y >= threshold] = 1
    y[y < threshold] = 0
    return y

def F1Measure(y_true, y_pred, threshold=0.5):
    y_binary = get_binary(y_pred, threshold)
    score = f1_score(y_true, y_binary, average = "macro")   

    return score

def train(mode, bert_layer, corp):
    
    model = build_model(bert_layer, max_len=256, classes = classes[corp][mode], activation = activation[mode])

    with open("../data/train_test/" + corp + "_train_" + mode + ".pkl", "rb") as f:
        X_train, y_train = pkl.load(f)

    checkpoint = tf.keras.callbacks.ModelCheckpoint('../models/' + corp + "_" + training + "_" + mode + '.h5', monitor='val_loss', save_best_only=True, verbose=1)
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    print("start training")
    t = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=200,
        callbacks=[checkpoint, earlystopping],
        batch_size=32, #32 works best so far
        verbose=1)
    print("Saving the model")

def crossVal(mode, threshold):
       
    with open("../data/train_test/" + corp + "_train_" + mode + ".pkl", "rb") as f:
        X, y = pkl.load(f)

    model_file = '../models/' + corp + '_' + training + "_" + mode + '_cv.h5'

    print("Start Cross-Validation")
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

    cvscores = []
    for train, test in kfold.split(X[0], reverse_onehot(y)): #potentially use CV folds as predictions to evaluate against chatGPT
        tf.keras.backend.clear_session() # remove any past model from session
        if os.path.isfile(model_file): # remove saved models from checkpoint
            os.remove(model_file)
        else:
            pass

        bert_layer = hub.KerasLayer(module_url, trainable=True)
        model = build_model(bert_layer, max_len=256, classes = classes[corp][mode], activation = activation[mode])
        checkpoint = tf.keras.callbacks.ModelCheckpoint(model_file, monitor='val_loss', save_best_only=True, verbose=1)
        earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
        
        X_train_cv = (X[0][train], X[1][train], X[2][train])
        y_train_cv = tf.gather(y, train)
        X_test_cv = (X[0][test], X[1][test], X[2][test])
        y_test_cv = tf.gather(y, test)
        t = model.fit(
            X_train_cv, y_train_cv,
            validation_data = (X_test_cv, y_test_cv),
            epochs=200,
            callbacks=[checkpoint, earlystopping],
            batch_size=32, #32 works best so far
            verbose=1)

        #load best model from training
        tf.keras.backend.clear_session() 
        model = load_model(model_file, compile=True, custom_objects={"KerasLayer": bert_layer})
        y_pred_val = model.predict(X_test_cv)
        score = F1Measure(y_test_cv, y_pred_val, threshold)
        cvscores.append(score * 100)
        print("%s: %.2f%%" % ("F1-Score (macro average)", score*100))
        
        score2 = f1_score(y_test_cv, get_binary(y_pred_val, threshold), average=None)
        print(score2.round(3)*100)        
        
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

def reverse_onehot(onehot_data):
    # onehot_data assumed to be channel last
    data_copy = np.zeros(onehot_data.shape[:-1])
    for c in range(onehot_data.shape[-1]):
        img_c = onehot_data[..., c]
        data_copy[img_c == 1] = c
    return data_copy
    
module_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

## General Parameters

In [7]:
# choose MFRC as corpus (can be changed to run on other corpora as necessary)
# choose to run on full MFT dimensions (see prepare_data for different ways of categorizing the moral values)
# Choose between training=eval for determining train/validation accuracy (e.g., when optimizing parameters) and training=normal to train the model

corp = "mfrc"
mode = "full"
training = "eval"
threshold = 0.3 #change this value when using eval (decision rule for classification; can impact accuracy)

## Train/Eval

In [13]:
if training == "eval": # determine best model using CV
    crossVal(mode, threshold)
elif training == "normal": # regular training for test sample (against chatGPT)
    train(mode, bert_layer, corp)
else:
    pass

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_mask (InputLayer)        [(None, 256)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 256)]        0           []                               
                                                                                                  
 input_word_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'pooled_output': (  17488641    ['input_mask[0][0]',             
                                None, 256),                       'segment_ids[0][0]',      

2023-09-04 00:23:05.176826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?,256]
	 [[{{node Placeholder_1}}]]
2023-09-04 00:23:05.176885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32 and shape [?,256]
	 [[{{node Placeholder_2}}]]


336/336 [==============================] - ETA: 0s - loss: 0.2485 - precision_2: 0.7322 - recall_2: 0.5445
Epoch 1: val_loss improved from inf to 0.20754, saving model to ../models/mfrc_normal_full2.h5
336/336 [==============================] - 73s 163ms/step - loss: 0.2485 - precision_2: 0.7322 - recall_2: 0.5445 - val_loss: 0.2075 - val_precision_2: 0.8349 - val_recall_2: 0.5727
Epoch 2/200
336/336 [==============================] - ETA: 0s - loss: 0.2098 - precision_2: 0.8037 - recall_2: 0.5799
Epoch 2: val_loss improved from 0.20754 to 0.19980, saving model to ../models/mfrc_normal_full2.h5
336/336 [==============================] - 41s 120ms/step - loss: 0.2098 - precision_2: 0.8037 - recall_2: 0.5799 - val_loss: 0.1998 - val_precision_2: 0.8248 - val_recall_2: 0.5773
Epoch 3/200
336/336 [==============================] - ETA: 0s - loss: 0.1930 - precision_2: 0.8127 - recall_2: 0.6108
Epoch 3: val_loss improved from 0.19980 to 0.18446, saving model to ../models/mfrc_normal_full2.h